In [2]:
!pip install basemap

     |████████████████████████████████| 862 kB 5.1 MB/s 
     |████████████████████████████████| 30.5 MB 1.7 MB/s 
     |████████████████████████████████| 219 kB 40.3 MB/s 
     |████████████████████████████████| 6.3 MB 39.5 MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37324 sha256=8c1b0064565a4801b135a94a09331a0bce7f8085dd622b2479005a20cc21e487
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os
import numpy as np
from array import array
sys.path.insert(0, '/content/drive/My Drive/dop_notebooks')
import matplotlib
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as patches
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import BSTUtilities
import BSTCalcMethods
import BSTFileMethods
import BSTPlotMethods
import BSTMethods

import datetime
import statistics
import dateutil.parser


from tabulate import tabulate

import time
from time import sleep  #   Added a pause of 30 seconds between downloads

import math

In [4]:
def get_circle_master_catalog(Latitude,Longitude,Radius,Magnitude):

#   This code will use shapely.py to filter the worldwide catalog
#       into a rectangular region

    settings_params = BSTUtilities.get_settings()
    region_type = settings_params[0]
    location = settings_params[3]

    data_string = 'Building circle catalog for region around ' + location + '...'
    
    if region_type == 'Circle':
        completeness_mag = float(settings_params[1])
        earthquake_depth = float(settings_params[2])
        settings_params[1] = completeness_mag
        settings_params[2] = earthquake_depth
        BSTUtilities.save_settings(settings_params)

    output_file_name = "/content/drive/MyDrive/dop_notebooks/USGSData/USGS_circle.master.catalog"
    output_file = open(output_file_name, "w")
    output_file.close()

    #   Compute vertices that define the circle around the city
    
    lng_circle_dg, lat_circle_dg = BSTUtilities.createCircleAroundWithRadius(Latitude, Longitude, Radius)
    
    number_polygon_vertices = len(lng_circle_dg)
    
    point_list = []

    for i in range(number_polygon_vertices):
        point_list.append((float(lat_circle_dg[i]),float(lng_circle_dg[i])))
    
    polygon = Polygon(point_list)
    
#   print point_list
#       

    input_file_name     = "/content/drive/MyDrive/dop_notebooks/USGSData/USGS_WorldWide.catalog"
    input_file          =  open(input_file_name, "r")

    output_file_name = "/content/drive/MyDrive/dop_notebooks/USGSData/USGS_circle.master.catalog"
    output_file = open(output_file_name, "w")
    
    for line in input_file:
        items = line.strip().split()
        dep    = items[6]
        mag    = items[5]
        eq_lat = items[4]
        eq_lng = items[3]
        
        point = Point((float(eq_lat),float(eq_lng)))
        
        if (float(dep) <= float(earthquake_depth) and float(mag) >= float(completeness_mag) and polygon.contains(point) == True):
#           print items[0],items[1],items[2],items[3],items[4],items[5],items[6]
            print(items[0],items[1],items[2],items[3],items[4],items[5],items[6], file=output_file)
        
    output_file.close()
    input_file.close()

    return None
    
    #############################################################

    # Set the catalog parameters for circular region

def get_circle_catalog(Circle_Lat, Circle_Lng, Radius_float, Rebuild_flag, \
        circle_catalog_date_start, circle_catalog_date_end):

    settings_params = BSTUtilities.get_settings()
    region_type = settings_params[0]
    location = settings_params[3]

    if region_type == 'Circle':
        completeness_mag = float(settings_params[1])
        earthquake_depth = float(settings_params[2])
        
#     print 'completeness_mag: ', completeness_mag
#     print ''

    if Rebuild_flag == 'OFF':
        data_string = 'Building catalog for  M>' + str(completeness_mag) + ' for small circle around ' + location + '...'
        
    if Rebuild_flag == 'ON':
            data_string = 'Rebuilding catalog for M>' + str(completeness_mag) + ' for small circle around ' + location 
    
    print('------------------------------------------')
    print('')
    print(data_string)
    print('')
    print('------------------------------------------')
#     print ''

    #############################################################
    
    get_circle_master_catalog(Circle_Lat, Circle_Lng, Radius_float, completeness_mag)
    
    # Open the master output file to read
    input_circle_master = open("/content/drive/MyDrive/dop_notebooks/USGSData/USGS_circle.master.catalog", "r")

    #   Write output record file

    # Open the output file
    output = open("/content/drive/MyDrive/dop_notebooks/USGSData/USGS_circle.catalog", "w")

    # Format the data

    mag_array   =   []
    date_array  =   []
    time_array  =   []
    year_array  =   []
    depth_array =   []
    lat_array   =   []
    lng_array   =   []

    i=-1
    for line in input_circle_master:
        i+=1
        items = line.strip().split()

        try:
        
            lat                 = items[4]
            lon                 = items[3]
            dep                 = items[6]
            mag                 = items[5]
            date_string         = items[0]
            time_string         = items[1]
            ts                  = items[2]

            mag_array.append(mag)           #   List of magnitudes
            date_array.append(date_string)
            time_array.append(time_string)
            year_array.append(ts)
            depth_array.append(dep)
            lat_array.append(lat)
            lng_array.append(lon)
            
            if float(mag) >= 6.49:
                print('')
                print('Data for events >= 6.5: ')
                print(items)
                print('')

            if float(dep) <= float(earthquake_depth) and float(ts) >= circle_catalog_date_start and \
                    float(ts) <= circle_catalog_date_end:
                output.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n" % (date_string, time_string, ts, lon, lat, mag, dep))

        except:
            pass

    # Finalize the output file
    input_circle_master.close()
    output.close()

    return mag_array, date_array, time_array, year_array, depth_array, lat_array, lng_array
    
def read_circle_catalog():

    mag_array   =   []
    date_array  =   []
    time_array  =   []
    year_array  =   []
    depth_array =   []
    lat_array   =   []
    lng_array   =   []

    data_file = open("/content/drive/MyDrive/dop_notebooks/USGSData/USGS_circle.catalog","r")
    
    for line in data_file:
        items = line.strip().split()
        
        try:
        
            lat                 = items[4]
            lon                 = items[3]
            dep                 = items[6]
            mag                 = items[5]
            date_string         = items[0]
            time_string         = items[1]
            ts                  = items[2]

            mag_array.append(mag)           #   List of magnitudes
            date_array.append(date_string)
            time_array.append(time_string)
            year_array.append(ts)
            depth_array.append(dep)
            lat_array.append(lat)
            lng_array.append(lon)
            
        except:
            pass
    
    data_file.close()  

    return mag_array, date_array, time_array, year_array, depth_array, lat_array, lng_array

    ######################################################################

In [5]:
#datetime.date.today().strftime("%F")
date_info = str(datetime.date.today())
date_data = date_info.split('-')
current_date = str(date_data[0]) + '/' + str(date_data[1]) + '/' + str(date_data[2])
current_time_info = str(datetime.datetime.now())
current_time = current_time_info.split(' ')[1]
current_time = str(current_time.split('.')[0])
#
#   ------------------------------------------------------------

#   SoCal
maxlng = -110.0
minlng = -126.
maxlat =  40.0
minlat = 28.0

depth = 100.0                       #   Defines maximum earthquake depth (variable)
km = 2000.0                         #   Radius Defines regional length scale for statistics in km (default=2000)

start_date = "1975/01/01"       #   Events downloaded occurred after this date
cutoff_date = 2100.0

circle_catalog_date_start = 1975.0
circle_catalog_date_end   = 2020.999

city_circle = 400.0     #   <-- Was 400.0
completeness_mag_uniform = 2.99     #   This is for the Worldwide catalog
completeness_mag = 3.29             #   This is for the catalog used in the calculations

# cluster_percent = 0.985  #  ~ 3*sigma -> 0.985 for SoCal with 400 km circle M>2.99 centered on SoCal/USA,33.5,-118.5
                                #   0.992 for Kyoto, 350 km centered on Kyoto, M>3.99
sd_factor  = 25        #   This is CLF,it filters cluster outliersResults seem to be insensitive to this value in the range 50-150
                        #       at a value of median ratio_limit for 600 km radius
                    
cutoff_start_year = 1984.0
cutoff_start_date = "1984/1/1"

cutoff_radgyr = 1.0         #   Cutoff radius for plotting burst centroids on the map (Km)
offset_year = 3.0           #   Value of time in years prior to M7 earthquakes we are using for centroid plot

Large_EQ = 'Landers'
Large_EQ = 'Hector Mine'
Large_EQ = 'El Major Cucupah'
Large_EQ = 'Ridgecrest'

#   .......................................................

N_Steps = 23

ratio_limit = 0.2   #   0.2 seems to give interesting results for the centroid_dev method

                    #   (We also find that this value produces the same radius of gyration for all M>7 earthquakes.
                    #       Physical reason this is good is that if this failure radius is to have some physical meaning,
                    #       it should be the same for all large earthquakes This is our fundamental hypothesis, that
                    #       the minimum radius of gyration has physical meaning.

                    
plot_burst_size = 30  #   large number means no burst plots

burst_print_flag = 'ON'

#   ------------------------------------------------------------

WW_Download = 'OFF'  #   Turn this off if you want to speed up code by not using the data download

if WW_Download == 'ON':
    print('')
    print('Downloading the World Wide catalog...')
    print('')
    BSTFileMethods.get_worldwide_catalog(maxlat, minlat, maxlng, minlng, completeness_mag_uniform, start_date)
#
#   -----------------------------------
#
   
    #.....................................
    #
    # Read pre-defined locations file
    #
    #.....................................

input_file = open("/content/drive/MyDrive/dop_notebooks/city_locations.txt", "r")
i=0
for line in input_file:
    i +=  1
input_file.close()  # Put the file pointer back at top

number_cities = i

print('')
print('Number of Cities: ', number_cities)


Number of Cities:  1


In [7]:
def get_regional_rate(completeness_mag, earthquake_depth, cutoff_start_year):

    number_earthquakes = 0

    input_file = open("/content/drive/MyDrive/dop_notebooks/USGSData/USGS_WorldWide.catalog", "r")

    for line in input_file:
        items = line.strip().split()

        date                = items[0]
        time                = items[1]
        year                = items[2]
        dep                 = items[6]
        mag                 = items[5]
            
        if float(mag) >= float(completeness_mag) and float(dep) <= float(earthquake_depth) and float(year) >= float(cutoff_start_year):
            number_earthquakes += 1
                
    input_file.close()

    time_interval = (float(year) - cutoff_start_year)*365.0
    
    regional_rate = float(number_earthquakes)/time_interval
    
    return regional_rate
    
def EMA_weights(N_events, N_Steps):

    #   This method computes the weights for the Exponential Weighted Average (EMA)

    alpha = 2./float((N_Steps+1))

    #   time_series_list is the time series of floating point values
    #       arranged in order of first element in list being earliest

    assert 0 < alpha <= 1
    
    weights = []
    
    #   Define the weights
    
    for i in range(0,N_events):
        weight_i = (1.0-alpha)**i
        weights.append(weight_i)
        
    sum_weights = sum(weights)
    weights =  [i/sum_weights for i in weights]
     
    return weights
    

In [ ]:
def map_epicenters(NELat, NELng, SWLat, SWLng, MagLo, Location):
   
    lat_0_center = 0.5*(NELat + SWLat)
    lon_0_center = 0.5*(NELng + SWLng)


    llcrnrlat = SWLat
    llcrnrlon  = SWLng
    urcrnrlat  = NELat
    urcrnrlon  = NELng

    m = Basemap(projection='cyl',llcrnrlat=SWLat, urcrnrlat=NELat,
            llcrnrlon=SWLng, urcrnrlon=NELng, lat_0=lat_0_center, lon_0=lat_0_center, lat_ts=20, resolution='h')

    m.drawmeridians(np.arange(0,360,8),labels=[0,0,0,1])
    m.drawparallels(np.arange(-90,90,4),labels=[1,0,0,0])

    #   m.shadedrelief()
    #   m.bluemarble()
    #   m.etopo()
    m.drawcoastlines()
    m.drawcountries()
    m.drawstates()
    #m.drawrivers()

    m.etopo()

    settings_params = BSTUtilities.get_settings()

    region_type = settings_params[0]

    #   -------------------------------------------

    if region_type == 'Circle':
        Polygon_Location = 'None'

        Circle_Location = settings_params[3]
        earthquake_depth = float(settings_params[2])

        if Circle_Location != 'None':
            earthquake_depth = float(settings_params[2])
            CircleCenterLat = float(settings_params[4])
            CircleCenterLng = float(settings_params[5])
            CircleRadius    = float(settings_params[6])

         #   -------------------------------------------

    if region_type == 'Polygon':
        Circle_Location = 'None'

        number_polygon_vertices = (len(settings_params)-4)/2

        Polygon_Location = settings_params[3]
        earthquake_depth = float(settings_params[2])

        if Polygon_Location != 'None':
            earthquake_depth = float(settings_params[2])

        x_poly = []
        y_poly = []

        for j in range(0,number_polygon_vertices):
            y_poly.append(settings_params[4+2*j])
            x_poly.append(settings_params[5+2*j])

    #   Close the polygon

        y_poly.append(settings_params[4])
        x_poly.append(settings_params[5])

        for i in range(0,len(x_poly)):
            x_poly[i] = float(x_poly[i])
            y_poly[i] = float(y_poly[i])

  
    #   -------------------------------------------


    #   Open input file and find the number of lines in the file

    input_file = open("EQ_Working_Catalog.txt", "r")

    i=0
    for line in input_file:
        i       +=  1

    input_file.close()  # Put the file pointer back at top

    number_eqs=i

    #   Open input file again

    input_file = open("EQ_Working_Catalog.txt", "r")

    # Loop over lines and extract variables of interest

    # Create arrays of length i filled with zeros

    indx_string         =   ["" for x in range(i)]
    yrs                 =   zeros(i)
    lng                 =   zeros(i)
    lat                 =   zeros(i)
    mag                 =   zeros(i)
    dep                 =   zeros(i)
    time_string         =   ["" for x in range(i)]
    date_string         =   ["" for x in range(i)]

    # Loop over lines and extract variables of interest

    i=-1
    for line in input_file:
        line = line.strip()
        data = line.split()
        data_array = np.asarray(data)

        i       +=  1

        indx_string[i] =   data_array[0]
        date_string[i] =   data_array[1]
        time_string[i] =   data_array[2]

        yrs[i]     =    float(data_array[3])
        lng[i]     =    float(data_array[4])
        lat[i]     =    float(data_array[5])
        mag[i]     =    float(data_array[6])
        dep[i]     =    float(data_array[7])

    input_file.close()  # Put the file pointer back at top

    z_limit = len(lat)
    range_limit = z_limit - 1

    print_text_1 = '     Found ' + str(number_eqs) + ' earthquakes having M>' + str(MagLo) + ' around ' + Location
    print_text_2 = '     Occurring from: ' + date_string[0] + ' ' + time_string[0] 
    print_text_3 = '               thru: ' + date_string[number_eqs-1] + ' ' + time_string[number_eqs - 1]
    print_text_4 = '     at depths < ' + str(settings_params[2]) + ' km'
    #print ' '
    #print print_text_1
    #print ' '
    #print print_text_2
    #print print_text_3
    #print ' '

    #.................................................................

    for z in range(z_limit):
        x,y = m(lng,lat)

    #   -----------------------
        if mag[z] >= 5.0:
            mark_size = 3
        if mag[z] >= 5.5:
            mark_size = 4
        if mag[z] >= 6:
            mark_size = 6
        if mag[z] >= 6.5:
            mark_size = 8
        if mag[z] >= 7:
            mark_size = 10
        if mag[z] >= 7.5:
            mark_size = 12
        if mag[z] >= 8.0:
            mark_size = 14
        if mag[z] >= 8.5:
            mark_size = 16
    #   ----------------------
            
    #   m.plot(x[z], y[z], "ro", mfc='none', mec='r', markeredgewidth=2.0, ms=mark_size[z])
        m.plot(x[z], y[z], "ro", ms=mark_size)
        m.plot(x[z], y[z], "o", ms=mark_size, fillstyle='none', color='k', lw=0.4)
        

    if (Circle_Location != 'None'):
#        x_circle_dg, y_circle_dg = draw_big_circle(CircleCenterLat, CircleCenterLng, CircleRadius)
        x_circle_dg, y_circle_dg = BSTUtilities.createCircleAroundWithRadius(CircleCenterLat, CircleCenterLng, CircleRadius)
        
        m.plot(x_circle_dg, y_circle_dg, "b-", lw=1.3)

    if (Polygon_Location != 'None'):
        m.plot(x_poly, y_poly, "b-", lw=1.6)


    #   ------------------------------------------------------
    #   Plot 150 km circle at latest event

    lat_latest = lat[i]
    lng_latest = lng[i]
    x,y = m(lng_latest,lat_latest)

    #   Draw a circle of radius 200 km

    Radius = 200.0  # km
    mark_size = int(155.0 * Radius/200.0)   #  approx at mid latitudes

    #   mark_size = 155
    #   m.plot(x,y, "o", mfc='none', ms=mark_size)

    #   ------------------------------------------------------

    sfv     =   (NELat-SWLat)/12.0
    sfh     =   (NELng-SWLng)/16.0

    sfh     =   sfh * 0.9
    sfht    =   sfh * 1.1  # move the text over a bit more

    lat=[llcrnrlat - 0.30*sfv, llcrnrlat + 0.25*sfv,llcrnrlat + 0.80*sfv, llcrnrlat + 1.30*sfv, llcrnrlat + 1.80*sfv, llcrnrlat + 2.40*sfv, llcrnrlat + 3.1*sfv, llcrnrlat + 3.95*sfv]
    lng=[urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht, urcrnrlon + 1.2*sfht]
    x,y = m(lng,lat)

    latc=[llcrnrlat - 0.25*sfv, llcrnrlat + 0.30*sfv,llcrnrlat + 0.85*sfv, llcrnrlat + 1.375*sfv, llcrnrlat + 1.90*sfv, llcrnrlat + 2.50*sfv, llcrnrlat + 3.2*sfv, llcrnrlat + 4.0*sfv]
    lngc=[urcrnrlon + 0.72*sfh, urcrnrlon + 0.72*sfh, urcrnrlon + 0.73*sfh, urcrnrlon + 0.74*sfh, urcrnrlon + 0.76*sfh, urcrnrlon + 0.78*sfh, urcrnrlon + 0.80*sfh, urcrnrlon + 0.80*sfh]
    xc,yc = m(lngc,latc)

    mag_value=['5.0+','5.5+','6+','6.5+','7+','7.5+', '8+', '8.5+']

    mark_size = [3, 4, 6, 8, 10, 12, 14, 16]

    for z in range(8):
        plt.text(x[z],y[z],mag_value[z], fontsize=10,)
        m.plot(xc[z], yc[z], "ro", ms=mark_size[z], clip_on=False)
        m.plot(xc[z], yc[z], "ro", ms=mark_size[z], clip_on=False, fillstyle='none', color='k', lw=0.4)

    City = Location.split('_')
    Number_Spaces = len(City)

    Location_Actual = City[0] + ' '
    for i in range(1,Number_Spaces):
        Location_Actual += City[i] + ' '

    City_Location = Location_Actual[:-1]

    if region_type == 'Circle':

        if Circle_Location == 'None':
            SupTitle_text = 'Earthquake Epicenters for M>' + str(MagLo) + ' near ' + Location

        if Circle_Location != 'None':
            Circle_Location_actual = Circle_Location.split('-')
            SupTitle_text = 'Earthquakes M>' + str(MagLo) + ' near ' + City_Location

    if region_type == 'Polygon':

        if Polygon_Location == 'None':
            SupTitle_text = 'Earthquake Epicenters for M>' + str(MagLo) + ' in ' + Location + ' at Depth < ' + str(earthquake_depth) +  'km'

        if Polygon_Location != 'None':
            Polygon_Location_actual = Polygon_Location.split('-')
            SupTitle_text = 'Earthquakes M>' + str(MagLo) + ' in ' + Location + ' at Depth < ' + str(earthquake_depth) + ' km'

    plt.suptitle(SupTitle_text, fontsize=14)

    Title_text = 'From: ' + date_string[0] + ' To: ' + date_string[range_limit] + ' R<' + str(int(CircleRadius)) \
            + ' km, D<'+ str(int(earthquake_depth)) +' km' 
    plt.title(Title_text, fontsize=12)

    #m.fillcontinents(color='#ffe8a0',lake_color='#e6e6ff')
    m.drawmapboundary(fill_color='#e6e6ff')

    figure_name = './Data/' + Circle_Location + '_Seismicity.png'

    matplotlib.pyplot.savefig(figure_name,dpi=600)
    #matplotlib.pyplot.savefig('./Data/Seismicity-Map.jpg')
    matplotlib.pyplot.close('all')

#    print ' '
#    print '     Close plot window to continue...'

    #plt.show()

    return None

    ######################################################################
    
def map_epicenters_local(NELat_local, NELng_local, SWLat_local, SWLng_local, completeness_mag, MagLo, Location):
   
    lat_0_center = 0.5*(NELat_local + SWLat_local)
    lon_0_center = 0.5*(NELng_local + SWLng_local)


    llcrnrlat  = SWLat_local
    llcrnrlon  = SWLng_local
    urcrnrlat  = NELat_local
    urcrnrlon  = NELng_local

    m = Basemap(projection='cyl',llcrnrlat=SWLat_local, urcrnrlat=NELat_local,
            llcrnrlon=SWLng_local, urcrnrlon=NELng_local, lat_0=lat_0_center, lon_0=lat_0_center, lat_ts=20, resolution='h')

    m.drawmeridians(np.arange(0,360,4),labels=[0,0,0,1], color='k', textcolor='k', linewidth=0.4)
    m.drawparallels(np.arange(-90,90,2),labels=[1,0,0,0],color='k', textcolor='k', linewidth=0.4)

    m.fillcontinents(color='tan', lake_color='aqua')
    m.drawcoastlines(linewidth=0.4, linestyle='solid', color='k',)
    m.drawcountries()
    m.drawstates()
    m.drawrivers()

    settings_params = get_settings()

    region_type = settings_params[0]

    #   -------------------------------------------

    if region_type == 'Circle':
        Polygon_Location = 'None'

        Circle_Location = settings_params[3]
        earthquake_depth = float(settings_params[2])

        if Circle_Location != 'None':
            earthquake_depth = float(settings_params[2])
            CircleCenterLat = float(settings_params[4])
            CircleCenterLng = float(settings_params[5])
            CircleRadius    = float(settings_params[6])

         #   -------------------------------------------

    if region_type == 'Polygon':
        Circle_Location = 'None'

        number_polygon_vertices = (len(settings_params)-4)/2

        Polygon_Location = settings_params[3]
        earthquake_depth = float(settings_params[2])

        if Polygon_Location != 'None':
            earthquake_depth = float(settings_params[2])

        x_poly = []
        y_poly = []

        for j in range(0,number_polygon_vertices):
            y_poly.append(settings_params[4+2*j])
            x_poly.append(settings_params[5+2*j])

    #   Close the polygon

        y_poly.append(settings_params[4])
        x_poly.append(settings_params[5])

        for i in range(0,len(x_poly)):
            x_poly[i] = float(x_poly[i])
            y_poly[i] = float(y_poly[i])

  
   #   -------------------------------------------
   #
   #    Re-download the data in the circular region for the local seismicity plot

    settings_params = get_settings()

    earthquake_depth    =   float(settings_params[2])
    Circle_Location     =   settings_params[3]
    Circle_Lat          =   float(settings_params[4])
    Circle_Lng          =   float(settings_params[5])
    Radius_float        =   float(settings_params[6])

    mag_array, date_array, time_array, year_array, depth_array, lat_array, lng_array = read_circle_catalog()

   #    Reverse the magnitude array for the magnitudes of events in the circular region

    mag_array_reversed  =   list(reversed(mag_array))
    date_array_reversed =   list(reversed(date_array))
    time_array_reversed =   list(reversed(time_array))
    year_array_reversed =   list(reversed(year_array))
    depth_array_reversed=   list(reversed(depth_array))
    lat_array_reversed  =   list(reversed(lat_array))
    lng_array_reversed  =   list(reversed(lng_array))

#    print lat_array_reversed[:]
#    print lng_array_reversed[:]
    

   #    Now find the number of small events within the circle since last large event

    todays_count = 0    

    last_eq_mag     = '(No Data)'
    last_eq_date    = '(No Data)'
    last_eq_time    = '(No Data)'
    last_eq_year    = '(No Data)'
    last_eq_lat     = '(No Data)'
    last_eq_lng     = '(No Data)'
    last_eq_depth   = '(No Data)'

   #.................................................................
   #
   #   This next bit finds the data on the last big earthquake

    mag_flag = 0
    for i in range(0,len(mag_array_reversed)):
        if float(mag_array_reversed[i]) >= MagLo and (mag_flag == 0) and (float(depth_array_reversed[i]) <= earthquake_depth):

            last_eq_mag         = str(mag_array_reversed[i])
            last_eq_date        = date_array_reversed[i]
            last_eq_time        = time_array_reversed[i]  
            last_eq_year        = year_array_reversed[i]
            last_eq_lat         = lat_array_reversed[i]
            last_eq_lng         = lng_array_reversed[i]
            last_eq_depth       = depth_array_reversed[i]

            mag_flag = 1
        if float(mag_array_reversed[i]) < MagLo and mag_flag == 0:
            todays_count += 1

   #.................................................................
   #
   #    Write catalog of most recent small earthquakes in the circle

    working_file_local = open("EQ_Working_Catalog_Local.txt", "w")    

    for i in range(0,todays_count):
        working_file_local.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\n" % \
                (i+1, date_array_reversed[i], time_array_reversed[i], year_array_reversed[i], lng_array_reversed[i], \
                lat_array_reversed[i], mag_array_reversed[i], depth_array_reversed[i]))

    if (last_eq_mag != '(No Data)'):
        working_file_local.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t\n" % (i+2, last_eq_date, last_eq_time, last_eq_year, \
                last_eq_lng, last_eq_lat, last_eq_mag, last_eq_depth))

    working_file_local.close()


   #.................................................................



    #   Open input file and find the number of lines in the file

    input_file = open("EQ_Working_Catalog_Local.txt", "r")

    i=0
    for line in input_file:
        i       +=  1

    input_file.close()  # Put the file pointer back at top

    number_eqs=i

    #   Open input file again

    input_file = open("EQ_Working_Catalog_Local.txt", "r")

    # Loop over lines and extract variables of interest

    # Create arrays of length i filled with zeros

    indx_string         =   ["" for x in range(i)]
    yrs                 =   zeros(i)
    lng                 =   zeros(i)
    lat                 =   zeros(i)
    mag                 =   zeros(i)
    dep                 =   zeros(i)
    time_string         =   ["" for x in range(i)]
    date_string         =   ["" for x in range(i)]

    # Loop over lines and extract variables of interest

    i=-1
    for line in input_file:
        line = line.strip()
        data = line.split()
        data_array = np.asarray(data)

        i       +=  1

        indx_string[i] =   data_array[0]
        date_string[i] =   data_array[1]
        time_string[i] =   data_array[2]

        yrs[i]     =    float(data_array[3])
        lng[i]     =    float(data_array[4])
        lat[i]     =    float(data_array[5])
        mag[i]     =    float(data_array[6])
        dep[i]     =    float(data_array[7])

    input_file.close()  # Put the file pointer back at top

    z_limit = len(lat)
    range_limit = z_limit - 1

    if z_limit > 0:
        print_text_1 = '     Found ' + str(number_eqs) + ' earthquakes having M>' + str(MagLo) + ' around ' + Location
        print_text_2 = '     Occurring from: ' + date_string[0] + ' ' + time_string[0] 
        print_text_3 = '               thru: ' + date_string[number_eqs-1] + ' ' + time_string[number_eqs - 1]
        print_text_4 = '     at depths < ' + str(settings_params[2]) + ' km'


    #.................................................................

    color_start =   0.0
    color_stop  =   1.0

    #   z_limit is the number of points

    cm_subsection = linspace(color_start, color_stop, z_limit) 
    colors = [ cm.rainbow(x) for x in cm_subsection ]

    mag_diff = 0.1*(float(MagLo) - float(completeness_mag))

    mag1 = completeness_mag
    mag2 = completeness_mag + 1.0*mag_diff
    mag3 = completeness_mag + 2.0*mag_diff
    mag4 = completeness_mag + 3.0*mag_diff
    mag5 = completeness_mag + 4.0*mag_diff
    mag6 = completeness_mag + 5.0*mag_diff
    mag7 = completeness_mag + 6.0*mag_diff
    mag8 = completeness_mag + 7.0*mag_diff
    mag9 = completeness_mag + 8.0*mag_diff
    mag10= completeness_mag + 9.0*mag_diff
    mag11= MagLo

    if z_limit >0:

        for z in range(z_limit):
            x,y = m(lng,lat)

    #   -----------------------

            w = z_limit-z-1

            if mag[w] >= mag1:
                mark_size = 4
            if mag[w] >= mag2:
                mark_size = 5
            if mag[w] >= mag3:
                mark_size = 6
            if mag[w] >= mag4:
                mark_size = 7
            if mag[w] >= mag5:
                mark_size = 8
            if mag[w] >= mag6:
                mark_size = 9
            if mag[w] >= mag7:
                mark_size = 10
            if mag[w] >= mag8:
                mark_size = 11
            if mag[w] >= mag9:
                mark_size = 12
            if mag[w] >= mag10:
                mark_size = 13
            if mag[w] >= mag11:
                mark_size = 20
                
    #   ----------------------
     
            m.plot(x[w], y[w], "o", ms=mark_size, color=colors[z])
            m.plot(x[w], y[w], "o", ms=mark_size, fillstyle='none', color='k', lw=0.4)
            

    if (Circle_Location != 'None'):
        x_circle_dg, y_circle_dg = draw_big_circle(CircleCenterLat, CircleCenterLng, CircleRadius)
        x_circle_dg, y_circle_dg = BSTUtilities.createCircleAroundWithRadius(CircleCenterLat, CircleCenterLng, CircleRadius)
        
        m.plot(x_circle_dg, y_circle_dg, "b-", lw=1.3)

    if (Polygon_Location != 'None'):
        m.plot(x_poly, y_poly, "b-", lw=1.6)

    #   ------------------------------------------------------

    sfv     =   (NELat_local-SWLat_local)/12.0
    sfh     =   (NELng_local-SWLng_local)/16.0

    sfh     =   sfh * 0.9
    sfht    =   sfh * 1.35  # move the text over a bit more

    lat=[llcrnrlat, llcrnrlat + 0.775*sfv]
    lng=[urcrnrlon + 1.25*sfht, urcrnrlon + 1.25*sfht]

    x,y = m(lng,lat)

    latc=[llcrnrlat + 0.15*sfv, llcrnrlat + 0.9*sfv]
    lngc=[urcrnrlon + 0.85*sfh, urcrnrlon + 0.85*sfh]
    
    xc,yc = m(lngc,latc)

    mag_value=[str(completeness_mag),str(MagLo)+'+']

    mark_size = [4, 20]

    for z in range(2):
        plt.text(x[z],y[z],mag_value[z], fontsize=10)
        m.plot(xc[z], yc[z], "ro", ms=mark_size[z], clip_on=False)
        m.plot(xc[z], yc[z], "ro", ms=mark_size[z], clip_on=False, fillstyle='none', color='k', lw=0.6)

    City = Location.split('_')
    Number_Spaces = len(City)

    Location_Actual = City[0] + ' '
    for i in range(1,Number_Spaces):
        Location_Actual += City[i] + ' '

    City_Location = Location_Actual[:-1]

    if region_type == 'Circle':

        if Circle_Location == 'None':
            SupTitle_text = 'Earthquake Epicenters for M>' + str(MagLo) + ' near ' + Location

        if Circle_Location != 'None':
            Circle_Location_actual = Circle_Location.split('-')
            SupTitle_text = 'Small Earthquakes M>' + str(completeness_mag) + ' near ' + City_Location 

    if region_type == 'Polygon':

        if Polygon_Location == 'None':
            SupTitle_text = 'Earthquake Epicenters for M>' + str(MagLo) + ' in ' + Location + ' at Depth < ' + str(earthquake_depth) +  'km'

        if Polygon_Location != 'None':
            Polygon_Location_actual = Polygon_Location.split('-')
            SupTitle_text = 'Earthquakes M>' + str(completeness_mag) + ' in ' + Location + ' at Depth < ' + str(earthquake_depth) + ' km'

    plt.suptitle(SupTitle_text, fontsize=14)

    if z_limit > 0:
        Title_text = 'Since M'+ str(mag[range_limit]) + ' on ' + date_string[range_limit] + ' ' + \
                ' To: ' + date_string[0] + ' R<'+ str(int(CircleRadius)) + ' km, D<'+ str(int(earthquake_depth)) +' km'
        plt.title(Title_text, fontsize=12)

    #m.fillcontinents(color='#ffe8a0',lake_color='#e6e6ff')
    m.drawmapboundary(fill_color='#e6e6ff')

    figure_name = './Data/' + Circle_Location + '_Seismicity_Local.png'

    matplotlib.pyplot.savefig(figure_name,dpi=600)
    matplotlib.pyplot.close('all')

#    print ' '
#    print '     Close plot window to continue...'

    #plt.show()

    return None


In [6]:
  # Create arrays of length i filled with zeros

Location_file   = ["" for x in range(i)]
Country_file    = ["" for x in range(i)]
Center_Lat_file = np.zeros(i)
Center_Lng_file = np.zeros(i)

MinLat_file     = np.zeros(i)
MaxLat_file     = np.zeros(i)
MinLng_file     = np.zeros(i)
MaxLng_file     = np.zeros(i)

MinLat_local_file     = np.zeros(i)
MaxLat_local_file     = np.zeros(i)
MinLng_local_file     = np.zeros(i)
MaxLng_local_file     = np.zeros(i)

    #   Compute the size of the region and store for later use
    #   Start by assuming a circle of radius "km" around city (lat,long)

input_file = open("/content/drive/MyDrive/dop_notebooks/city_locations.txt", "r")

i=-1
for line in input_file:
    i+=1
    line    = line.strip()
    items   = line.split(',')

    #   Read in the city, its lat and long

    items_array = np.asarray(items)

    city_country            = items_array[0]

    city                    = city_country.split('/')[0]
    country                 = city_country.split('/')[1]

    Location_file[i]   = city
    Center_Lat_file[i] = items_array[1]
    Center_Lng_file[i] = items_array[2]
    Country_file[i]         = city_country.split('/')[1]
 
    delta_Lng = BSTUtilities.change_in_longitude(Center_Lat_file[i], km)
    delta_Lat = BSTUtilities.change_in_latitude(km)


    delta_local_Lng = BSTUtilities.change_in_longitude(Center_Lat_file[i], city_circle)
    delta_local_Lat = BSTUtilities.change_in_latitude(city_circle)

    MinLat_file[i]     = Center_Lat_file[i] - delta_Lat
    MaxLat_file[i]     = Center_Lat_file[i] + delta_Lat
    MinLng_file[i]     = Center_Lng_file[i] - delta_Lng
    MaxLng_file[i]     = Center_Lng_file[i] + delta_Lng

    MinLat_local_file[i]     = Center_Lat_file[i] - delta_local_Lat
    MaxLat_local_file[i]     = Center_Lat_file[i] + delta_local_Lat
    MinLng_local_file[i]     = Center_Lng_file[i] - delta_local_Lng
    MaxLng_local_file[i]     = Center_Lng_file[i] + delta_local_Lng

input_file.close()  # Put the file pointer back at top

    #.....................................
    
for i in range(0,number_cities):

    Circle_Location     =   Location_file[i]
    Circle_Lat          =   Center_Lat_file[i]
    Circle_Lng          =   Center_Lng_file[i]
    Radius_float        =   city_circle
    earthquake_depth    =   depth
    region_type         =   'Circle'

    small_radius = str(Radius_float)
    length_scale   = str(2.*km)
    depth        = str(earthquake_depth)

    settings_params     =   []
    settings_params.append(region_type)
    settings_params.append(completeness_mag)
    settings_params.append(earthquake_depth)
    settings_params.append(Circle_Location)
    settings_params.append(Circle_Lat)
    settings_params.append(Circle_Lng)
    settings_params.append(Radius_float)

    BSTUtilities.save_settings(settings_params)

    settings_params = BSTUtilities.get_settings()

    Location    = Location_file[i]
    SWLat       = MinLat_file[i]
    NELat       = MaxLat_file[i]
    SWLng       = MinLng_file[i]
    NELng       = MaxLng_file[i]

    SWLat_local       = MinLat_local_file[i] - 0.25
    NELat_local       = MaxLat_local_file[i] + 0.25
    SWLng_local       = MinLng_local_file[i] - 0.25
    NELng_local       = MaxLng_local_file[i] + 0.25
    
    # Build the regional catalog from the World Wide catalog
    # Build the circle catalog from the World Wide catalog
    Rebuild_flag = 'OFF'
    
    get_circle_catalog(Circle_Lat,Circle_Lng,Radius_float,Rebuild_flag, circle_catalog_date_start, circle_catalog_date_end)
   
    # Plot Local EQ Epicenters on a Map in US
    if (i<number_cities and Country_file[i] == 'USA'):

        settings_params = BSTUtilities.get_settings()
        settings_params[1] = completeness_mag
        BSTUtilities.save_settings(settings_params)
        
        # Build the circle catalog again from the World Wide catalog but with potentially smaller completeness mag
        Rebuild_flag = 'ON'
        
        regional_rate = get_regional_rate(completeness_mag, depth, cutoff_start_year)
        
        get_circle_catalog(Circle_Lat,Circle_Lng,Radius_float,Rebuild_flag, \
                circle_catalog_date_start, circle_catalog_date_end) 

    # Plot Local EQ Epicenters on a Map in US
        BSTMethods.map_epicenters_local(NELat_local, NELng_local, SWLat_local, SWLng_local, completeness_mag, Location)

        burst_index, number_events, min_daily_events, burst_min_size, mag, lat, lng, date, time, years = \
                    BSTPlotMethods.plot_swarm_event_counts\
                            (completeness_mag, Location, sd_factor, regional_rate, burst_print_flag)
#                     
 
        BSTPlotMethods.plot_radgyr_time(burst_min_size, completeness_mag, Location, \
                circle_catalog_date_start, circle_catalog_date_end, burst_index, mag, lat, lng, date, time, years)
                
        N_Steps, radgyr_list, year_swarm, index_swarm, centroid_lat, centroid_lng  = \
                BSTPlotMethods.plot_radgyr_EMA_time(burst_min_size, completeness_mag, Location,  N_Steps,\
                circle_catalog_date_start, circle_catalog_date_end, burst_index, mag, lat, lng, date, time, years, \
                ratio_limit, sd_factor, cutoff_start_year, regional_rate)
                
        BSTPlotMethods.plot_cluster_density_rgyr(burst_min_size, completeness_mag, Location, N_Steps,\
                circle_catalog_date_start, circle_catalog_date_end, burst_index, mag, lat, lng, date, time, years, \
                ratio_limit, cutoff_start_year)
                
        BSTPlotMethods.map_swarm_centroids(Location, burst_index, mag, lat, lng, date, time, years, cutoff_start_year, cutoff_start_date)
        
        BSTPlotMethods.map_swarm_centroids_alternate(Location, burst_index, mag, lat, lng, date, time, years, \
                cutoff_start_year, cutoff_start_date, cutoff_radgyr, sd_factor, ratio_limit)
                
        BSTPlotMethods.map_swarm_centroids_dev1(Location, burst_index, mag, lat, lng, date, time, years, \
                cutoff_start_year, cutoff_start_date, cutoff_radgyr, sd_factor, ratio_limit, offset_year, Large_EQ)
                
        #   This is probably the most important function
        BSTPlotMethods.plot_radgyr_EMA_time_dev(burst_min_size, completeness_mag, Location, N_Steps,\
                circle_catalog_date_start, circle_catalog_date_end, burst_index, mag, lat, lng, date, time, years, \
                ratio_limit, sd_factor, cutoff_start_year, regional_rate)
                

#   -----------------------------------------------------------------

        if i == 0:
            for j in range(len(burst_index)):
                burst_number = j
                if len(burst_index[j]) >=  plot_burst_size and len(burst_index[j]) <= 15.*plot_burst_size:
                    BSTPlotMethods.map_swarm_epicenters_one_burst(completeness_mag, Location, N_Steps, \
                            burst_index, burst_number, mag, lat, lng, date, time, years)

#   -----------------------------------------------------------------

------------------------------------------

Building catalog for  M>3.29 for small circle around Los_Angeles...

------------------------------------------

Data for events >= 6.5: 
['1983/05/02', '23:42:38.060', '1983.333333', '-120.312', '36.2316667', '6.7', '9.578']


Data for events >= 6.5: 
['1987/11/24', '13:15:56.710', '1987.896175', '-115.852', '33.015', '6.6', '11.181']


Data for events >= 6.5: 
['1992/06/28', '11:57:34.130', '1992.491803', '-116.437', '34.2', '7.3', '-0.097']


Data for events >= 6.5: 
['1994/01/17', '12:30:55.390', '1994.046448', '-118.537', '34.213', '6.7', '18.202']


Data for events >= 6.5: 
['1999/10/16', '09:46:44.460', '1999.789617', '-116.265', '34.6033333', '7.1', '13.73']


Data for events >= 6.5: 
['2003/12/22', '19:15:56.240', '2003.972678', '-121.1005', '35.7005', '6.5', '8.382']


Data for events >= 6.5: 
['2010/04/04', '22:40:42.360', '2010.256831', '-115.2953333', '32.2861667', '7.2', '9.987']


Data for events >= 6.5: 
['2019/07/06', '03:19:

FileNotFoundError: ignored